# Code generation

**pyecsca** can generate C implementations of ECC crypto for several microprocessor targets, which
are defined by the `Platform` enum.

In [1]:
from pyecsca.codegen.common import Platform


Platform.names()

['HOST', 'XMEGA', 'STM32F0', 'STM32F3']

To generate an implementation we need an actual configuration which the implementation should
implement, which is stored inside the `Configuration` and `DeviceConfiguration` classes. 

In [2]:
from typing import get_args
from pyecsca.codegen.common import DeviceConfiguration
from dataclasses import fields

for field in fields(DeviceConfiguration):
	name = field.name
	tp = field.type
	doc = tp.__doc__
	if get_args(field.type):
		doc = get_args(field.type)[0].__doc__
	if tp == bool:
		doc = ""
	print(name, tp)
	print("   ", doc)
	if hasattr(tp, "names"):
		for enum_name in tp.names():
			print("       ", enum_name)
	print()
()

model <class 'pyecsca.ec.model.CurveModel'>
    A model(form) of an elliptic curve.

coords <class 'pyecsca.ec.coordinates.CoordinateModel'>
    A coordinate system for a particular model(form) of an elliptic curve.

formulas typing.Set[pyecsca.ec.formula.Formula]
    A formula operating on points.

scalarmult <class 'pyecsca.ec.mult.ScalarMultiplier'>
    
    A scalar multiplication algorithm.

    :param short_circuit: Whether the use of formulas will be guarded by short-circuit on inputs
                          of the point at infinity.
    :param formulas: Formulas this instance will use.
    

hash_type <enum 'HashType'>
    Hash algorithm used in ECDH and ECDSA.
        NONE
        SHA1
        SHA224
        SHA256
        SHA384
        SHA512

mod_rand <enum 'RandomMod'>
    Method of sampling a uniform integer modulo order.
        SAMPLE
        REDUCE

mult <enum 'Multiplication'>
    Base multiplication algorithm to use.
        TOOM_COOK
        KARATSUBA
        COMB

()

The `DeviceConfiguration` class contains a few additional attributes apart from those
in the `Configuration` class: `platform`, `keygen`, `ecdh` and `ecdsa`.

The `platform` attribute defines for which target the implementation
should be built. The other boolean attributes specify whether particular
functionality should be implemented and enabled in the implementation.

## Generating

We will first create a `DeviceConfiguration`, which we will then generate and build.

In [3]:
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.configuration import *

platform = Platform.STM32F3
hash_type = HashType.SHA1
mod_rand = RandomMod.REDUCE
mult = Multiplication.BASE
sqr = Squaring.BASE
red = Reduction.BASE

model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
add = coords.formulas["add-1998-cmo"]
dbl = coords.formulas["dbl-1998-cmo"]
scl = coords.formulas["z"]
formulas = [add, dbl, scl]
scalarmult = LTRMultiplier(add, dbl, scl)

config = DeviceConfiguration(model, coords, formulas, scalarmult, 
							 hash_type, mod_rand, mult, sqr, red,
							 platform, True, True, True)

config

DeviceConfiguration(model=ShortWeierstrassModel(), coords=EFDCoordinateModel("projective" on short Weierstrass curves), formulas=[AdditionEFDFormula(add-1998-cmo for EFDCoordinateModel("projective" on short Weierstrass curves)), DoublingEFDFormula(dbl-1998-cmo for EFDCoordinateModel("projective" on short Weierstrass curves)), ScalingEFDFormula(z for EFDCoordinateModel("projective" on short Weierstrass curves))], scalarmult=<pyecsca.ec.mult.LTRMultiplier object at 0x7f071c2cd4c0>, hash_type=HASH_SHA1, mod_rand=MOD_RAND_REDUCE, mult=MUL_BASE, sqr=SQR_BASE, red=RED_BASE, platform=CW308_STM32F3, keygen=True, ecdh=True, ecdsa=True)

Now we can render the configuration, which will generate the source files into a
randomly created temporary directory, and return the path to the directory as
well as names of the elf and hex files which will be built in that directory.

In [4]:
from pyecsca.codegen.builder import render

directory, elf_name, hex_name = render(config)

print(directory)

/tmp/tmp1wtoeyhk


## Building

When we have the implementation rendered, we can build it using make.

In [5]:
from subprocess import run

res = run(["make"], cwd=directory, capture_output=True)
print(res.stdout.decode())

rm -f -- pyecsca-codegen-CW308_STM32F3.hex
rm -f -- pyecsca-codegen-CW308_STM32F3.eep
rm -f -- pyecsca-codegen-CW308_STM32F3.cof
rm -f -- pyecsca-codegen-CW308_STM32F3.elf
rm -f -- pyecsca-codegen-CW308_STM32F3.map
rm -f -- pyecsca-codegen-CW308_STM32F3.sym
rm -f -- pyecsca-codegen-CW308_STM32F3.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s bn/bn.s asn1/asn1.s hash/hash.s prng/prng.s gen/curve.s gen/formula_add.s gen/formula_dbl.s gen/formulas.s gen/formula_scl.s gen/mult.s gen/point.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s
rm -f -- main.d bn/bn.d asn1/asn1.d hash/hash.d prng/prng.d gen/curve.d gen/formula_add.d gen/formula_dbl.d gen/formulas.d gen/formula_scl.d gen/mult.d gen/point.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d
rm -f -- main.i bn/bn.i asn1/asn1.i hash/hash.i prng/prng.i gen/curve.i gen/formula_add.i gen/formula_dbl.i gen/formulas.i gen/formula_scl.i gen/mult.i gen/point.i simpleserial.i stm32f3_hal.

Now the files `elf_name` and `hex_name` in the directory contain the ELF file and HEX file built.


In [6]:
res = run(["file", elf_name], cwd=directory, capture_output=True)
print(res.stdout.decode())

res = run(["file", hex_name], cwd=directory, capture_output=True)
print(res.stdout.decode())

pyecsca-codegen-CW308_STM32F3.elf: ELF 32-bit LSB executable, ARM, EABI5 version 1 (SYSV), statically linked, with debug_info, not stripped

pyecsca-codegen-CW308_STM32F3.hex: ASCII text, with CRLF line terminators



## Running

We will now run key generation using the generated implementation on
the `STM32F3` target.

In [7]:
from pyecsca.codegen.client import DeviceTarget
from pyecsca.ec.params import get_params

params = get_params("secg", "secp128r1", "projective")
target = DeviceTarget(params.curve.model, params.curve.coordinate_model, Platform.STM32F3, timeout=10000)

Flash the implementation.

In [8]:
from os.path import join

target.flash(join(directory, hex_name))


Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 29271 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 29271 bytes
Serial baud rate = 38400


True

Run the key generation.

In [9]:
target.connect()
target.set_params(params)
priv, pub = target.generate()
target.disconnect()

Serial baud rate = 38400
Serial baud rate = 115200


And we can check that the generated keypair is valid.

In [10]:
print(params.curve.is_on_curve(pub))

print(priv)
print(pub)
print(params.curve.affine_multiply(params.generator.to_affine(), priv))

True
189359030463681349816366610478560039051
[x=122835813094999453922649270086793500655, y=326514220558629293368386081113307347349]
[x=122835813094999453922649270086793500655, y=326514220558629293368386081113307347349]


## Running on the host

We can also run the implementation on the host.

In [11]:
config = DeviceConfiguration(model, coords, formulas, scalarmult, 
							 hash_type, mod_rand, mult, sqr, red,
							 Platform.HOST, True, True, True)

directory, elf_name, hex_name = render(config)
res = run(["make"], cwd=directory, capture_output=True)
print(res.stdout.decode())

rm -f -- pyecsca-codegen-HOST.hex
rm -f -- pyecsca-codegen-HOST.eep
rm -f -- pyecsca-codegen-HOST.cof
rm -f -- pyecsca-codegen-HOST.elf
rm -f -- pyecsca-codegen-HOST.map
rm -f -- pyecsca-codegen-HOST.sym
rm -f -- pyecsca-codegen-HOST.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- main.s bn/bn.s asn1/asn1.s hash/hash.s prng/prng.s gen/curve.s gen/formula_add.s gen/formula_dbl.s gen/formulas.s gen/formula_scl.s gen/mult.s gen/point.s simpleserial.s uart.s host_hal.s
rm -f -- main.d bn/bn.d asn1/asn1.d hash/hash.d prng/prng.d gen/curve.d gen/formula_add.d gen/formula_dbl.d gen/formulas.d gen/formula_scl.d gen/mult.d gen/point.d simpleserial.d uart.d host_hal.d
rm -f -- main.i bn/bn.i asn1/asn1.i hash/hash.i prng/prng.i gen/curve.i gen/formula_add.i gen/formula_dbl.i gen/formulas.i gen/formula_scl.i gen/mult.i gen/point.i simpleserial.i uart.i host_hal.i
mkdir objdir objdir/hash objdir/prng objdir/asn1 objdir/bn objdir/gen
mkdir .dep
.
-------- begin --------
gcc (Arch Linux 9.2.1+

In [12]:
from pyecsca.codegen.client import HostTarget
from os.path import join

target = HostTarget(params.curve.model, params.curve.coordinate_model, binary=join(directory, elf_name))

In [13]:
target.connect()
target.set_params(params)
priv, pub = target.generate()
target.disconnect()

In [14]:
print(params.curve.is_on_curve(pub))

print(priv)
print(pub)
print(params.curve.affine_multiply(params.generator.to_affine(), priv))

True
162938999268550597445809790209592423458
[x=111810799217268384317536017529141796945, y=309320541414531923178173772704935971498]
[x=111810799217268384317536017529141796945, y=309320541414531923178173772704935971498]
